# Pathway Impact Explorer
Interactive explorer for cross-omics pathway convergence and feature-level details.



In [ ]:
import os
import json
import math
import base64
import io
from typing import List, Dict, Any

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML
from ipycytoscape import CytoscapeWidget
import requests

API_BASE = os.environ.get("API_BASE", "http://localhost:8000/api/v1")
TIMEOUT = 10

status_output = widgets.Output()
network_output = widgets.Output()
detail_output = widgets.Output()
download_output = widgets.Output()


def log_status(message: str, color: str = "#333"):
    with status_output:
        print(message)



In [ ]:
# Demo data fallback

demo_pathways = [
    {
        "id": "p1",
        "name": "NF-kB signaling",
        "p_value": 1e-4,
        "matched_by_omics": ["gene", "protein"],
        "features": {
            "gene": ["NFKB1", "RELA", "IL6"],
            "protein": ["IKBKB", "CHUK"],
        },
    },
    {
        "id": "p2",
        "name": "TCA cycle",
        "p_value": 5e-3,
        "matched_by_omics": ["metabolite"],
        "features": {
            "metabolite": ["Citrate", "Succinate", "Malate"],
        },
    },
    {
        "id": "p3",
        "name": "Lipid metabolism",
        "p_value": 2e-2,
        "matched_by_omics": ["lipid", "gene"],
        "features": {
            "lipid": ["Cer(d18:1/16:0)", "PC(16:0/18:1)"],
            "gene": ["SREBF1", "FASN"],
        },
    },
]



In [ ]:
def api_get(path: str, params: Dict[str, Any] | None = None):
    try:
        resp = requests.get(f"{API_BASE}{path}", params=params, timeout=TIMEOUT)
        resp.raise_for_status()
        return resp.json()
    except Exception as exc:  # noqa: BLE001
        log_status(f"API GET failed: {exc}")
        return None


def load_pathways(program_id: str | None = None, p_thresh: float = 0.05):
    params = {"p_value": p_thresh}
    if program_id:
        params["program_id"] = program_id
    data = api_get("/pathways/enrich", params=params)
    if data is None:
        return demo_pathways
    return data


def make_download_link(df: pd.DataFrame, filename: str = "pathways.csv"):
    buf = io.StringIO()
    df.to_csv(buf, index=False)
    b64 = base64.b64encode(buf.getvalue().encode()).decode()
    return HTML(f"<a download='{filename}' href='data:text/csv;base64,{b64}'>⬇️ Download CSV</a>")



In [ ]:
# UI widgets
program_dropdown = widgets.Dropdown(options=["Demo Program"], description="Program")
omics_options = [
    widgets.Checkbox(value=True, description="Gene", indent=False),
    widgets.Checkbox(value=True, description="Protein", indent=False),
    widgets.Checkbox(value=True, description="Metabolite", indent=False),
    widgets.Checkbox(value=True, description="Lipid", indent=False),
]
pvalue_slider = widgets.FloatSlider(value=0.05, min=0.0001, max=0.1, step=0.0001, description="p-value")
refresh_btn = widgets.Button(description="Refresh", button_style="info")

controls = widgets.VBox([
    widgets.HBox([program_dropdown, pvalue_slider, refresh_btn]),
    widgets.HBox(omics_options),
])



In [ ]:
def selected_omics():
    mapping = {
        "Gene": "gene",
        "Protein": "protein",
        "Metabolite": "metabolite",
        "Lipid": "lipid",
    }
    return [mapping[o.description] for o in omics_options if o.value]


def filter_by_omics(pathways: List[Dict[str, Any]], selected_types: List[str]):
    if not selected_types:
        return []
    return [p for p in pathways if set(p.get("matched_by_omics", [])) & set(selected_types)]


def build_cytoscape_graph(pathways: List[Dict[str, Any]]):
    cy = CytoscapeWidget()
    cy.set_style([
        {"selector": "node", "style": {"label": "data(label)", "font-size": 10}},
        {"selector": "edge", "style": {"width": "data(weight)", "line-color": "#bbb"}},
    ])
    nodes = []
    edges = []

    # Build pathway nodes
    for p in pathways:
        size = max(20, min(80, -math.log10(p.get("p_value", 1e-3)) * 15))
        nodes.append({
            "data": {
                "id": p["id"],
                "label": p.get("name", ""),
                "p_value": p.get("p_value"),
                "matched_by_omics": ", ".join(p.get("matched_by_omics", [])),
            },
            "classes": "pathway",
            "style": {"width": size, "height": size, "background-color": "#2a9d8f"},
        })

    # Build edges based on shared features between pathways
    for i, p1 in enumerate(pathways):
        feats1 = p1.get("features", {})
        flat1 = {f for flist in feats1.values() for f in flist}
        for p2 in pathways[i + 1 :]:
            feats2 = p2.get("features", {})
            flat2 = {f for flist in feats2.values() for f in flist}
            shared = flat1 & flat2
            if shared:
                edges.append(
                    {
                        "data": {
                            "source": p1["id"],
                            "target": p2["id"],
                            "weight": max(1, len(shared)),
                            "label": f"{len(shared)} shared",
                        }
                    }
                )

    cy.graph.add_graph_from_json({"elements": {"nodes": nodes, "edges": edges}})
    return cy


def render_table(pathway: Dict[str, Any]):
    if not pathway:
        return HTML("<i>Select a pathway</i>")
    feats = pathway.get("features", {})
    rows = []
    for ftype, flist in feats.items():
        for f in flist:
            rows.append({"Omics": ftype, "Feature": f})
    df = pd.DataFrame(rows)
    return HTML(df.to_html(index=False))



In [ ]:
pathways = load_pathways()
selected_pathway = None
cy_widget = None


def export_graph_png():
    if cy_widget is None:
        return HTML("<i>No graph</i>")
    png_bytes = cy_widget.get_png()
    b64 = base64.b64encode(png_bytes).decode()
    return HTML(f"<a download='pathway_network.png' href='data:image/png;base64,{b64}'>⬇️ Download PNG</a>")


def refresh_graph(_=None):
    global pathways, cy_widget, selected_pathway
    p_thresh = pvalue_slider.value
    pathways = load_pathways(None, p_thresh)
    filtered = filter_by_omics(pathways, selected_omics())
    if network_output.outputs:
        network_output.clear_output()
    cy_widget = build_cytoscape_graph(filtered)

    def handle_click(event):
        global selected_pathway
        pid = event["data"]["id"]
        selected_pathway = next((p for p in filtered if p["id"] == pid), None)
        render_details()

    cy_widget.on("node", "click", handle_click)
    with network_output:
        display(cy_widget)
    render_details()


def render_details():
    detail_output.clear_output()
    download_output.clear_output()
    current = selected_pathway
    with detail_output:
        display(render_table(current))
        if current:
            df = pd.DataFrame(
                [
                    {"Pathway": current.get("name"), "p_value": current.get("p_value"), "Omics": ", ".join(current.get("matched_by_omics", []))}
                ]
            )
            display(make_download_link(df, filename="pathway_summary.csv"))
    with download_output:
        display(export_graph_png())


refresh_btn.on_click(refresh_graph)
for chk in omics_options:
    chk.observe(refresh_graph, names="value")

refresh_graph()



In [ ]:
display(widgets.VBox([
    controls,
    widgets.Label("Status"),
    status_output,
    widgets.Label("Network"),
    network_output,
    widgets.Label("Details"),
    detail_output,
    download_output,
]))

